In [ ]:
# The Main Function for APAD Project -- Cloth On Fly

# Author: Tianyi (Kelly) Zhang
#         Shikha Singh

# Creation Date: 07/21/2019

# Major: MSITM

In [ ]:
import sqlite3
import glob
import csv
from datetime import date, datetime,timedelta

### Databate Create

In [134]:
db = sqlite3.connect("ClothONFly.db")
cursor = db.cursor()

In [135]:
# cursor.execute('''
#               CREATE TABLE Users(User_ID INTEGER PRIMARY KEY AutoIncrement, User_Name Text NOT NULL UNIQUE, Password Text,
#               User_Type Text Not Null check(User_Type = "Admin" or User_Type = "Buyer" or User_Type = "Seller"), First_Name TEXT NOT NULL,
#               Last_Name TEXT NOT NULL, Email TEXT Not NUll, Address Text Not NUll, Phone_Num INTEGER Not Null)
#             ''')

# cursor.execute('''
#               CREATE TABLE Inventory_Items(Item_ID INTEGER PRIMARY KEY AutoIncrement, Brand_Name TEXT NOT NULL,
#               Cloth_Type TEXT NOT NULL, Size TEXT NOT NULL, Gender TEXT NOT NULL, Original_Price REAL NOT NULL,
#               Rental_Price REAL NOT NULL, Available_From DATE, Owner_ID INTEGER, Location TEXT NOT NULL, Cloth_Image BLOB NOT NULL, Deposit REAL NOT NULL,
#               Foreign Key(Owner_ID) References Users(User_ID))
#             ''')

# cursor.execute('''
#               CREATE TABLE Orders (Order_ID INTEGER PRIMARY KEY AutoIncrement, User_ID INTEGER NOT NULL, Item_ID INTEGER Not Null,
#               Delivery_Date Date Not Null, Return_Date Date Not Null, Actual_Return_Date Date, Shipping_Address Text Not Null,
#               Order_Status Text Not Null check(Order_Status = "Order Received(Shipment Pending)" or Order_Status = "Order Shipped" or Order_Status = "Order Returned" or Order_Status = "Return Received(Deposit Refunded)" or Order_Status = "Return Not Received(Deposit Withheld)"),
#               Shipping_Date Date, FOREIGN KEY(User_ID) REFERENCES Users(User_ID), FOREIGN KEY(Item_ID) References Inventory_Item(Item_ID))
#           ''')

In [136]:
# cursor.execute('''drop table Users''')
# cursor.execute('''drop table Inventory_Items''')
# cursor.execute('''drop table Orders''')

In [137]:
db.commit()
cursor.close()
db.close()

### utils file

In [138]:
ADMIN_ROLE_NAME = "Admin"
BUYER_ROLE_NAME = "Buyer"
SELLER_ROLE_NAME = "Seller"

ORDER_RECEIVED = "Order Received(Shipment Pending)"
ORDER_SHIPPED = "Order Shipped"
ORDER_RETURN_INIT = "Order Returned"
RETURN_RECEIVED = "Return Received(Deposit Refunded)"
RETURN_NOT_RECEIVED = "Return Not Received(Deposit Withheld)"

### activities.py

In [139]:
def register(user_name, password, user_role, first_name, last_name, email, address, phone_num) -> bool:

    if user_role.lower() == ADMIN_ROLE_NAME.lower():
        new_admin_user = Admin(user_name, password, first_name,
                               last_name, email, address, phone_num)
        write_user_into_database(ADMIN_ROLE_NAME, new_admin_user)
        print("Register succeeded! Returning to the main menu...\n")
        
    elif user_role.lower() == SELLER_ROLE_NAME.lower():
        new_seller_user = Seller(user_name, password, first_name,
                               last_name, email, address, phone_num)
        move_seller_to_pending(new_seller_user)
        print("Your registration is pending for authorization, thank you for your patience.")
        
    elif user_role.lower() == BUYER_ROLE_NAME.lower():
        print('This user is a buyer.')
        new_buyer_user = Buyer(user_name, password, first_name,
                                 last_name, email, address, phone_num)
        print('Writing the new buyer into db...')
        write_user_into_database(BUYER_ROLE_NAME, new_buyer_user)
        print("Register succeeded! Returning to the main menu...\n")
        
    else:
        print("Unknown role. Please try again.\n")
        return False

    return True

In [140]:
def check_duplicate(user_name):
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    try:
        cur_result = cursor.execute('''SELECT * FROM Users where User_Name = ? ''',(str(user_name),))
        db.commit()
        if len(cur_result.fetchall()) != 0:
            cursor.close()
            db.close()
            return True
        else:
            cursor.close()
            db.close()
            return False
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return True

In [141]:
def login(user_name, password):
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    
    try:
        cur_result = cursor.execute('''SELECT User_Type FROM Users where User_Name = ? AND Password = ?''', (str(user_name), str(password)))
        cur_result_all = cur_result.fetchall()
        db.commit()
        cursor.close()
        db.close()
        
        if len(cur_result_all) > 0:
            print("LogIn Succeeded!")
            for row in cur_result_all:
                return row[0]
        else:
            print("Unable to LogIn. Username or Password incorrect.")
       
    except Exception as e:
        cursor.close()
        db.close()
        print(e)

In [142]:
def write_user_into_database(user_role, user) -> bool:
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cursor.execute('''Insert into Users(User_Name, Password, User_Type, First_Name, Last_Name, Email, Address, Phone_Num, User_ID) VALUES(?,?,?,?,?,?,?,?,?)''',
                       (user.user_name, user.password, user_role, user.first_name, user.last_name, user.email, user.address, user.phone_num, None))
        db.commit()
        cursor.close()
        db.close()
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

In [143]:
PENDING_SELLER_LIST = list()
def move_seller_to_pending(seller) -> bool:
    PENDING_SELLER_LIST.append(seller)
    return True

In [144]:
def get_userid_from_username(user_name):
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cursor.execute('''SELECT User_ID FROM Users WHERE User_Name = ?''', (user_name,))
        db.commit()
        for row in cursor.fetchall():
            return row[0]
        cursor.close()
        db.close()
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return None

In [145]:
# This function takes fetches all orders in the system along with the item, buyer and seller details
# This function creates a csv file in the specified directory and writes the fetch result in the file
# The file is created with a date time stamp so that every time this function is executed a new file is created

def all_orders():
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    
    try:
        cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                i.Item_ID As "Product ID",i.Brand_Name AS "Product's Brand",i.Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender",
                                usr.First_Name as "Seller's first Name", usr.Last_Name as "Seller's Last Name", usr.Email As "Seller's Email",usr.Phone_Num AS "Seller's Contact"
                                from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID join Users usr on i.Owner_ID =usr.User_ID ''')

        with open(Seller.export_dir+'/'+"Orders_export_"+ str(datetime.now()) + ".csv", "w", newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow([i[0] for i in cursor.description])  # write headers
            csv_writer.writerows(cursor)
        
        db.commit()
        cursor.close()
        db.close()
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

### admin.py

In [146]:
class Admin:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num
    
    @staticmethod
    def authorize_sellers() -> bool:
        print("Hi Admin! Please check and authorize the following sellers for registration:")
        
        while (len(PENDING_SELLER_LIST) > 0):
            seller = PENDING_SELLER_LIST[0]
            print('PENDING_SELLER_LIST size: ' + str(len(PENDING_SELLER_LIST)) + "\n")
            # Notify the current admin about the seller's info
            print("Seller basic: \n", 
                  ((seller.user_name), (seller.password), (SELLER_ROLE_NAME), 
                   (seller.first_name), (seller.last_name), (seller.email), 
                   (seller.address), (seller.phone_num)))
            decision = input('Do you want to grant this seller permission to the system (Y/N)? ')
            if decision.lower() == 'y':
                print("Writing seller into database...")
                write_user_into_database(SELLER_ROLE_NAME, seller)
                print("Finished.\n")

            print("Removing the seller from the pending list...")
            PENDING_SELLER_LIST.remove(seller)
            print('PENDING_SELLER_LIST size: ' + str(len(PENDING_SELLER_LIST)) + "\n")
            
        print("Back to Admin Menu...\n")
        return True

    @staticmethod
    # Remove a user from the user table if needed
    def remove_user() -> bool:
        answer = input("Please input the user name you want to remove: ")
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''DELETE FROM Users 
                              WHERE User_Name = ?''',(answer,))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

        return False
    
    @staticmethod
    # This function takes fetches all orders in the system along with the item, buyer and seller details
    # This function creates a csv file in the specified directory and writes the fetch result in the file
    # The file is created with a date time stamp so that every time this function is executed a new file is created
    def all_orders():
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                i.Item_ID As "Product ID",i.Brand_Name AS "Product's Brand",i.Cloth_Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender",
                                usr.First_Name as "Seller's first Name", usr.Last_Name as "Seller's Last Name", usr.Email As "Seller's Email",usr.Phone_Num AS "Seller's Contact"
                                from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID join Users usr on i.Owner_ID =usr.User_ID ''')
            db.commit()

            with open(Seller.export_dir+'/'+"Orders_export_"+ str(datetime.now()) + ".csv", "w", newline='') as file:
                csv_writer = csv.writer(file)
                csv_writer.writerow([i[0] for i in cursor.description])  # write headers
                csv_writer.writerows(cursor)

            cursor.close()
            db.close()
            return True

        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

        

### seller.py

In [147]:
class Seller:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num
    
    image_dir = "Images/"
    export_dir = "Exports/"
    import_dir = "ImportFiles/"
        
    @staticmethod
    # This function add the clothing items in the database
    # The items are available for renting from the day they are added in the system. 
    # So we have used date.today() function to populate Available_From unless specified by seller explicitly
    def add_item(Brand_Name, Cloth_Type, Size, Gender, Original_Price, 
                 Rental_Price, Owner_ID, Location,
                 Cloth_Image, Deposit, Available_From) :
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        
        blobData = None
        
        try:
            # Checking the images in the specide image directory
            for image in glob.glob(Seller.image_dir+'/*'):
                # Checking if the image specified by the seller is available in the image directory
                if (str(image).replace(Seller.image_dir,'')) == Cloth_Image:
                    with open(image, 'rb') as file:
                        blobData = file.read()
        except Error as e:
            print("Could not read file. Check your file name and folder path")
 

        try:
            cursor.execute('''INSERT INTO Inventory_Items(Brand_Name, Cloth_Type, Size, 
                                Gender, Original_Price, Rental_Price, Owner_ID, Location, 
                                Cloth_Image, Deposit, Available_From) VALUES(?,?,?,?,?,?,?,?,?,?,?)''',
                               (Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, 
                                Owner_ID, Location, blobData, Deposit, Available_From))
            db.commit()
            cursor.close()
            db.close()
            return True

        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

        
    @staticmethod
    # This function removes the inventory item from the database
    def remove_item(Item_ID):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''Delete from Inventory_Items where Item_ID == ?''',(str(Item_ID)))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

    @staticmethod
    # This function updates any/all attributes of the inventory item in the database
    def update_item(Item_ID, Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, Owner_ID, Location, Cloth_Image, Deposit, Available_From):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        
        try:
            for image in glob.glob(Seller.image_dir + '/*'):
                if (str(image).replace(Seller.image_dir, '')) == Cloth_Image:
                    with open(image, 'rb') as file:
                        blobData = file.read()
        except Error as e:
            print("Could not read file. Check your file name and folder path")
        
        try:
            cursor.execute('''UPDATE Inventory_Items SET Brand_Name= ?, Cloth_Type = ?, Size = ?, Gender = ?, 
                            Original_Price =?, Rental_Price =?, Owner_ID=?, Location=?, Cloth_Image=?, Deposit=?,
                            Available_From=? WHERE Item_ID = ?''', 
                           (Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, Owner_ID, 
                            Location, blobData, Deposit, Available_From, Item_ID))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

    @staticmethod
    # This function will be called when the Seller clicks on ship item button. 
    # This will change the status of the order to "Order Shipped"
    # This will also update the shipping date to current date
    def ship_item(Order_ID):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''UPDATE Orders SET Order_Status = ?, Shipping_Date = ?
                              WHERE Order_ID = ?''', (ORDER_SHIPPED, date.today(), Order_ID))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

    @staticmethod
    # This function will list all the returned items for seller so that the seller can refund the respective deposits
    def view_orders_for_deposit_refund(Seller_ID):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        
        try:
            cursor.execute('''SELECT * FROM Orders
                              WHERE Return_Date >= Actual_Return_Date and Order_Status = ?''', (ORDER_RETURN_INIT))
        
            for item in cursor.fetchall():
                print("Order_ID: " + str(item[0]) + "; User_ID: " + str(item[1]) + "; Item_Id: " + str(item[2]) + 
                      "; Delivery_Date: " + str(item[3]) + "; Return_Date: " + str(item[4]) + "; Actual_Return_Date: " + str(item[5]) + 
                      "; Shipping_Address: " + str(item[6]) + "; Order_Status: " + str(item[7]) + "; Shipping_Date: " + str(item[8]) + ".\n")
                print("\n")
                
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
       
        cursor.close()
        db.close()
    
    @staticmethod
    # This function will list all the returned items for seller so that the seller can refund the respective deposits
    def view_orders_for_withholding_deposit(Seller_ID):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        
        try:
            cursor.execute('''SELECT * FROM Orders
                              WHERE Return_Date < ? and Actual_Return_Date = '' and 
                              Order_Status = ? ''',(date.today(), ORDER_SHIPPED))

            for item in cursor.fetchall():
                print("Order_ID: " + str(item[0]) + "; User_ID: " + str(item[1]) + "; Item_Id: " + str(item[2]) + 
                      "; Delivery_Date: " + str(item[3]) + "; Return_Date: " + str(item[4]) + "; Actual_Return_Date: " + str(item[5]) + 
                      "; Shipping_Address: " + str(item[6]) + "; Order_Status: " + str(item[7]) + "; Shipping_Date: " + str(item[8]) + ".\n")
                print("\n")
                
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
        
        db.commit()
        cursor.close()
        db.close()
    
    @staticmethod
    # This function will be called when the Seller clicks on refund deposit button. 
    # This function will update the order status to "Return Received(Deposit Refunded)" 
    # This function will update the item's Available From date as current date
    # This function will execute when the return has been initited by the buyer(order status is "Returned")
    # and return date is current day or a future date
    def refund_deposit(Order_ID):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        
        try:
            cursor.execute('''UPDATE Orders SET Order_Status = ?
                              WHERE Order_ID = ? and Return_Date >= Actual_Return_Date and Order_Status = ? ''', 
                             (RETURN_RECEIVED, str(Order_ID), ORDER_RETURN_INIT))
            cursor.execute('''SELECT Item_ID FROM Orders WHERE Order_ID = ?''',(Order_ID,))
            for row in cursor.fetchall():
                item = row[0]
                cursor.execute('''UPDATE Inventory_Items SET Available_From =? 
                WHERE Item_ID = ?''', (date.today(),item))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

    @staticmethod
    # This function will be called when the Seller clicks on withhold deposit button. 
    # This function will update the order status to "Return Not Received(Deposit Withheld)" 
    # This function will execute when the return has NOT been initited by the buyer(order status is NOT "Returned)
    # and return date is less that current date i.e. a past date
    def withhold_deposit(Order_ID):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        
        try:
            cursor.execute('''UPDATE Orders SET Order_Status = ?
                              WHERE Order_ID = ? and Return_Date < ? and Actual_Return_Date = '' and Order_Status != ? ''',
                             (RETURN_NOT_RECEIVED, int(Order_ID), date.today(), ORDER_RETURN_INIT))
            db.commit()
            cursor.close()
            db.close()
            return True
        
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
    
    @staticmethod
    # This function add the clothing items in the database in bulk
    # This function reads from the csv file provided by the Seller and adds each row of the csv as a new inventory item
    def bulk_item_upload(filename):
        # Checking the file in the specide import file directory
        for files in glob.glob(Seller.import_dir + '/*'):
            # Checking if the file specified by the seller is available in the directory
            if (str(files).replace(Seller.import_dir, '')) == filename:
                with open(Seller.import_dir + '/'+filename, 'r') as inputfile:
                    items = csv.DictReader(inputfile)

                    for item in items:
                        if item['AvailableFrom'] == '':
                            item['AvailableFrom'] = date.today()
                        Seller.add_item(item['Brand'], item['Type'], item['Size'], item['Gender'], 
                                          item['OriginalPrice'], item['RentalPrice'], item['OwnerID'], 
                                          item['Location'], item['ClothImage'], item['Deposit'], item['AvailableFrom'])
    
    @staticmethod
    # This function fetches the orders from a system along with the item and buyer for a specified seller
    # This function creates a csv file in the specified directory and writes the fetch result in the file
    # The file is created with the seller's username and date time stamp so that every time this function is executed a new file is created
    def orders_per_seller(Seller_ID):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        
        try:
            cursor.execute('''SELECT User_Name FROM Users WHERE User_ID = ?''',(Seller_ID,))
            for row in cursor.fetchall():
                seller = row[0]
                
            cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                            u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                            i.Item_ID As "Product ID",i.Brand_Name AS "Product's Brand",i.Cloth_Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender",
                                            i.Original_Price As "Product's Original Price", i.Rental_Price As "Product's Rental Price",
                                            i.Deposit As "Product's Deposit Amount"
                                            from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID 
                                            where i.Owner_ID = ? ''', (Seller_ID,))
            
            for item in cursor.fetchall():
                print("Order ID: " + str(item[0]) + "; Order Status: " + str(item[1]) + "; Delivery Date: " + str(item[2]) + 
                      "; Return Date: " + str(item[3]) + "; Shipping Address: " + str(item[4]) + "; Buyer's First Name: " + str(item[5]) + 
                      "; Buyer's LastName: " + str(item[6]) + "; Buyer's Email: " + str(item[7]) + "; Buyer's Contact: " + str(item[8]) + 
                      "; Product ID: " + str(item[9]) + "; Product's Brand: " + str(item[10]) + "; Product Type: " + str(item[11]) + 
                      "; Product's Size: " + str(item[12]) + "; Gender: " + str(item[13]) +
                      "; Product's Original Price: " + str(item[14]) + "; Product's Rental Price: " + str(item[15]) +
                      "; Product's Deposit Amount: " + str(item[16]) + 
                      ".\n")
                print("\n")

            print("\n *** An export of this is placed in the Exports directory with Seller's name and current datetimestamp***")

            cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                            u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                            i.Item_ID As "Product ID",i.Brand_Name AS "Product's Brand",i.Cloth_Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender",
                                            i.Original_Price As "Product's Original Price", i.Rental_Price As "Product's Rental Price",
                                            i.Deposit As "Product's Deposit Amount"
                                            from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID 
                                            where i.Owner_ID = ? ''', (Seller_ID,))
            db.commit()
            
            with open(Seller.export_dir + '/' + seller + "'s_Orders_export_" + str(datetime.now()) + ".csv", "w", newline='') as file:
                csv_writer = csv.writer(file)
                csv_writer.writerow([i[0] for i in cursor.description])  # write headers
                csv_writer.writerows(cursor)
               
            cursor.close()
            db.close()
            return True
        
        except Exception as e:
            print("Could not write to the file")
            cursor.close()
            db.close()
            print(e)
            return False
    

### buyer.py

In [148]:
class Buyer:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num

    @staticmethod    
    def view_item():
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''SELECT * FROM Inventory_Items''')
            result = cursor.fetchall()
            print("Inventory size: " + str(len(result)))
            for item in result:
                print("Item_ID: " + str(item[0]) + "; Brand_Name: " + str(item[1]) + "; Cloth_Type: " + str(item[2]) + 
                      "; Size: " + str(item[3]) + "; Gender: " + str(item[4]) + "; Original_Price: " + str(item[5]) + 
                      "; Rental_Price: " + str(item[6]) + "; Available_From: " + str(item[7]) + 
                      "; Owner_ID: " + str(item[8]) + "; Location: " + str(item[9]) + ".\n")
            print("\n")
            
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return
        
        cursor.close()
        db.close()
    
    @staticmethod
    def rent_item(id_num) -> bool:
        print("Renting item")
        item_id_num = input("Please input the item num you want to rent: ")
        shipping_address = input("Please input your shipping address: ")
        rent_days = int(input("Please input how many days you want to rent the item: "))
        delivery_date = date.today() + timedelta(days=7)
        total_price = Buyer.calculate_price(item_id_num, rent_days)
        print("The total price of your order  is: $" + str(total_price))
        response = input("\n Enter Y to continue with your order...\n") 
        if response.lower() == 'y' or response.lower() == 'yes':
            db = sqlite3.connect("ClothONFly.db")
            cursor = db.cursor()
            try:
                cursor.execute('''INSERT INTO Orders(Order_ID, User_ID, Item_ID, Delivery_Date, Return_Date, Shipping_Address, Order_Status, Shipping_Date)
                                  VALUES(?,?,?,?,?,?,?,?)
                               ''',(None, id_num, item_id_num, delivery_date, delivery_date + timedelta(days = rent_days), shipping_address, ORDER_RECEIVED, ""))
                db.commit()
                cursor.close()
                db.close()
                return True
            except Exception as e:
                cursor.close()
                db.close()
                print(e)
                return False
        else:
            print("Thank you for shopping with Cloth ON Fly!")
    
    @staticmethod
    def return_item() -> bool:
        print("Returning item")
        item_id_num = input("Please input the item num you want to return: ")
        # update Order_Status in Orders table
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''UPDATE Orders SET Order_Status = ?, Actual_Return_Date = ? WHERE Item_ID = ?''', (ORDER_RETURN_INIT, date.today(), item_id_num,))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
        return False
    
    @staticmethod
    def calculate_price(id_num,rent_days):
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''select Rental_Price,Deposit from Inventory_Items where Item_ID= ?''',(str(id_num),))
            db.commit()
            
            for row in cursor.fetchall():
                rental_price = row[0]
                deposit = row[1]
                total_price = float(deposit) + (int(rent_days) * int(rental_price))
                return total_price
            
            cursor.close()
            db.close()
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False
    

### main.py

In [149]:
def show_main_menu() -> bool:
    print("Hi there! Welcome to Cloth on Fly! What's Next?")
    print("[1] Register")
    print("[2] Log In")
    print("[3] Quit")
    main_answer = input("Input the action number here: ")
    # registration
    if main_answer == "1":
        print("Please input the following information to register!")
        
        user_name = input("Username: ")
        
        is_user_name_duplicate = check_duplicate(user_name)
        if (is_user_name_duplicate):
            print("This user name already exists in the database. Please use a different one for registration\n")
            return True
        
        password = input("Password: ")
        user_role = input("Do you want to register as admin/seller/buyer: ")
        first_name = input("First Name: ")
        last_name = input("Last Name: ")
        email = input("Email: ")
        address = input("Address: ")
        phone_num = int(input("Phone Number: "))
        
        register(user_name, password, user_role, first_name, last_name, email, address, phone_num)
        return True

    # login
    elif main_answer == "2":
        print("Hey Flyer! Please input your user name and password to log in: ")
        user_name = input("Username: ")
        password = input("Password: ")
        user_role = login(user_name, password)
        if user_role == None:
            print("User does not exist. Please try again.\n")
        else:
            print("User role is: " + user_role + "\n")
            user_menu_keep_looping = True
            while (user_menu_keep_looping):
                user_menu_keep_looping = show_user_menu(user_name, user_role)
        return True
    
    # quit
    elif main_answer == "3":
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [150]:
def show_user_menu(user_name, user_role):
    if (user_role == ADMIN_ROLE_NAME):
        return show_admin_menu(user_name)
    elif (user_role == SELLER_ROLE_NAME):
        return show_seller_menu(user_name)
    else:
        return show_buyer_menu(user_name)

In [151]:
def show_admin_menu(user_name):
    print(">>>>>>>> Admin Menu\n")
    print("[1] Authorize Sellers")
    print("[2] Remove An User")
    print("[3] Add An Item")
    print("[4] Remove An Item")
    print("[5] Update An Item")
    print("[6] View all orders")
    print("[7] Log Out")
    user_answer = input("Input the action number here: ")
    
    user_id = get_userid_from_username(user_name)
    
    if user_answer == '1':
        print("Retrieving pending sellers list...\n")
        Admin.authorize_sellers()
        print("Finished pending sellers review.\n")
        return True
    elif user_answer == '2':
        print("Removing an user...\n")
        Admin.remove_user()
        print("Finished user deletion.\n")
        return True
    elif user_answer == '3':
        print("Adding an item...\n")
        Brand_Name = input("Please input the brand of the cloth: ")
        Cloth_Type = input("Please input the type of the cloth: ")
        Size = input("Please input the size of the cloth: ")
        Gender = input("Please input the gender that fit for the cloth: ")
        Original_Price = input("Please input the original price of the cloth: ")
        Rental_Price = input("Please input the rental per day price for the cloth: ")
        Location = input("Please input the current location of the cloth: ")
        Cloth_Image = input("Please input the name of your cloth image")
        Deposit = input("Please input the deposit you want to hold for the cloth: ")
        Seller.add_item(Brand_Name, Cloth_Type, Size, Gender, Original_Price, 
                         Rental_Price, user_id, Location, Cloth_Image, Deposit, Available_From = date.today())
        print("Finished adding the item.\n")
        return True
    elif user_answer == '4':
        Item_ID = input("Please input the item ID that you want to remove: ")
        print("Removing an item...\n")
        Seller.remove_item(Item_ID)
        print("Finished removing the item.\n")
        return True
    elif user_answer == '5':
        print("Updating an item...\n")
        Item_ID = input("Please input the item ID that you want to update: ")
        Brand_Name = input("Please update the brand of the cloth: ")
        Cloth_Type = input("Please update the type of the cloth: ")
        Size = input("Please update the size of the cloth: ")
        Gender = input("Please update the gender that fit for the cloth: ")
        Original_Price = input("Please update the original price of the cloth: ")
        Rental_Price = input("Please update the rental per day price for the cloth: ")
        Owner_ID = input("Please update the current owner ID: ")
        Location = input("Please update the current location of the cloth: ")
        Cloth_Image = input("Please update the name of your cloth image: ")
        Deposit = input("Please input the deposit you want to hold for the cloth: ")
        Available_From = input("Please update the availablity date using the format xxxx-xx-xx: ")
        Seller.update_item(Item_ID, Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, Owner_ID, Location, Cloth_Image, Deposit, Available_From)
        print("Finished updating the item.\n")
        return True
    elif user_answer == '6':
        Admin.all_orders()
        print("A csv export of all order details is taken from the system and placed in /Exports folder with your username and current date and time stamp")
        return True
    elif user_answer == '7':
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [152]:
def show_seller_menu(user_name):
    print(">>>>>>>> Seller Menu\n")
    print("[1] Add An Item")
    print("[2] Remove An Item")
    print("[3] Update An Item")
    print("[4] Ship An Item")
    print("[5] Add items in bulk")
    print("[6] Show all orders for me")
    print("[7] View all returned orders for deposit refund")
    print("[8] Refund Deposit")
    print("[9] View all not returned orders for withholding deposit")
    print("[10] Withhold Deposit")
    print("[11] Log Out")
    user_answer = input("Input the action number here: ")
    
    user_id = get_userid_from_username(user_name)

    if user_answer == '1':
        print("Adding an item...\n")
        Brand_Name = input("Please input the brand of the cloth: ")
        Cloth_Type = input("Please input the type of the cloth: ")
        Size = input("Please input the size of the cloth: ")
        Gender = input("Please input the gender that fit for the cloth: ")
        Original_Price = input("Please input the original price of the cloth: ")
        Rental_Price = input("Please input the rental per day price for the cloth: ")
        Location = input("Please input the current location of the cloth: ")
        Cloth_Image = input("Please input the name of your cloth image: ")
        Deposit = input("Please input the deposit you want to hold for the cloth: ")
        Available_From = input("Please input the availablity date using the format xxxx-xx-xx: ")
        Seller.add_item(Brand_Name, Cloth_Type, Size, Gender, Original_Price, 
                         Rental_Price, user_id, Location, Cloth_Image, Deposit, Available_From)
        print("Finished adding the item.\n")
        return True
    elif user_answer == '2':
        Item_ID = input("Please input the item ID that you want to remove: ")
        print("Removing an item...\n")
        Seller.remove_item(Item_ID)
        print("Finished removing the item.\n")
        return True
    elif user_answer == '3':
        print("Updating an item...\n")
        Item_ID = input("Please input the item ID that you want to update: ")
        Brand_Name = input("Please update the brand of the cloth: ")
        Cloth_Type = input("Please update the type of the cloth: ")
        Size = input("Please update the size of the cloth: ")
        Gender = input("Please update the gender that fit for the cloth: ")
        Original_Price = input("Please update the original price of the cloth: ")
        Rental_Price = input("Please update the rental per day price for the cloth: ")
        Location = input("Please update the current location of the cloth: ")
        Cloth_Image = input("Please update the name of your cloth image: ")
        Deposit = input("Please input the deposit you want to hold for the cloth: ")
        Available_From = input("Please update the availablity date using the format xxxx-xx-xx: ")
        Seller.update_item(Item_ID, Brand_Name, Cloth_Type, Size, Gender, Original_Price, Rental_Price, user_id, Location, Cloth_Image, Deposit, Available_From)
        print("Finished updating the item.\n")
        return True
    elif user_answer == '4':
        print("Shipping an item...\n")
        Order_ID = input("Please input your order ID that you shipped out: ")
        Seller.ship_item(Order_ID)
        print("Order shipped...\n")
        return Tru5
    elif user_answer == '5':
        print("Enter all the item details in a csv file and place it in ImportFiles directory...\n")
        filename = input("Enter the name of the file to be loaded: ")
        print("Loading items...\n")
        Seller.bulk_item_upload(filename)
        return True
    elif user_answer == '6':
        print("Displaying all orders for you...\n")
        Seller.orders_per_seller(user_id)
        return True
    elif user_answer == '7':
        print("Displaying all returned orders for deposit refund...\n ")
        Seller.view_orders_for_deposit_refund(user_id)
        return True
    elif user_answer == '8':
        print("Deposit refund processing...\n")
        Order_ID = input("Enter the Order_ID that you want to refund:")
        Seller.refund_deposit(Order_ID)
        return True
    elif user_answer == '9':
        print("Displaying all not returned orders for withholding deposit...\n ")
        Seller.view_orders_for_withholding_deposit(user_id)
        return True
    elif user_answer == '10':
        print("Deposit withholding...\n")
        Order_ID = input("Enter the Order_ID that you want to withhold:")
        Seller.withhold_deposit(Order_ID)
        print("Deposit withheld...\n")
        return True
    elif user_answer == '11':
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True

In [153]:
def show_buyer_menu(user_name):
    print(">>>>>>>> Buyer Menu\n")
    print("[1] View Items")
    print("[2] Rent An Item")
    print("[3] Return An Item")
    print("[4] Log Out")
    user_answer = input("Input the action number here: ")

    if user_answer == '1':
        print("Checking inventory...\n")
        Buyer.view_item()
        return True
    elif user_answer == '2':
        print("Placing an order...\n")
        user_id = get_userid_from_username(user_name)
        Buyer.rent_item(user_id)
        print("Complete the order!\n")
        return True
    elif user_answer == '3':
        print("Item return processing...\n")
        Buyer.return_item()
        print("Return Initiated\n")
        return True
    elif user_answer == '4':
        print("Thanks for using Cloth ON Fly!\n")
        return False
    
    # input other options
    else:
        print("Input Unrecognized. Please try again!\n")
        return True
    

In [154]:
def main():
    keep_looping = True
    while (keep_looping):
        keep_looping = show_main_menu()
    
if __name__ == '__main__':
    main()

Hi there! Welcome to Cloth on Fly! What's Next?
[1] Register
[2] Log In
[3] Quit
Input the action number here: 2
Hey Flyer! Please input your user name and password to log in: 
Username: buyer_test
Password: testtest
LogIn Succeeded!
User role is: Buyer

>>>>>>>> Buyer Menu

[1] View Items
[2] Rent An Item
[3] Return An Item
[4] Log Out
Input the action number here: 1
Checking inventory...

Inventory size: 10
Item_ID: 1; Brand_Name: Zara; Cloth_Type: tshirt; Size: S; Gender: Female; Original_Price: 29.99; Rental_Price: 0.99; Available_From: 2019-07-28; Owner_ID: 3; Location: Austin.

Item_ID: 2; Brand_Name: Acne Studio; Cloth_Type: Sweater; Size: M; Gender: Unisex; Original_Price: 159.99; Rental_Price: 3.99; Available_From: 2019-07-28; Owner_ID: 3; Location: Los Angeles.

Item_ID: 4; Brand_Name: Levis; Cloth_Type: Jeans; Size: M; Gender: Male; Original_Price: 59.99; Rental_Price: 2.99; Available_From: 2019-08-20; Owner_ID: 3; Location: New York.

Item_ID: 5; Brand_Name: Calvin Clein; C

KeyboardInterrupt: 